In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/doubanbook-top250/doubanbook_top250_comments.txt
/kaggle/input/doubanbook-top250/doubanbook_top250_introduction.txt
/kaggle/input/doubanbook-top250/stopwords.txt


In [12]:

def handle_data(handle, cells):
    global headers
    if len(cells) != 6:
        pass
    elif headers == None:
        headers = cells
        print(headers)
    else:
        item = {}
        for i in range(6):
            item[headers[i]] = cells[i]
        handle(item)

def load_data(handle):
    global headers
    headers = None
    with open('/kaggle/input/doubanbook-top250/doubanbook_top250_comments.txt', 'r') as in_file:
        pre_line = [None]
        while True:
            line = in_file.readline()
            if not line:
                handle_data(handle,pre_line)
                break
            cells = line.strip().split('\t', 5)
            if pre_line[0] != cells[0] and len(cells) != 6 and len(pre_line) == 6:
                pre_line[5] += line
            else:
                handle_data(handle,pre_line)
                pre_line = cells


In [13]:
# 基于图书评论的推荐系统
# TF-IDF
import jieba

book_comments = {}
def assemble(item):
    book = item['book']
    if book == '': return
    global book_comments
    comments = book_comments.get(book)
    if comments == None:
        comments = []
        book_comments[book] = comments
    words = jieba.lcut(item['body'])
    comments.extend(words)
        

load_data(assemble)
print(len(book_comments))
print(book_comments.keys())

['book', 'id', 'star', 'time', 'likenum', 'body']
233
dict_keys(['天才在左 疯子在右', '1Q84 BOOK 1', '悲伤逆流成河', '恶意', 'Harry Potter and the Deathly Hallows', '长安乱', '苏菲的世界', '许三观卖血记', '1995-2005夏至未至', '盗墓笔记', '霍乱时期的爱情', '三生三世 十里桃花', '基督山伯爵', '小时代1.0折纸时代', '洛丽塔', '1Q84 BOOK 2', '第一次的亲密接触', '神雕侠侣', '一座城池', '茶花女', '当我谈跑步时我谈些什么', '明朝那些事儿（贰）', '人类简史', '一個人住第5年', '明朝那些事儿（肆）', '寻路中国', '我们台湾这些年', '1Q84 BOOK 3', '摆渡人', '明朝那些事儿（伍）', '骆驼祥子', '盗墓笔记3', '麦琪的礼物', '格林童话全集', '水仙已乘鲤鱼去', '历史深处的忧虑', '金锁记', '草样年华', '刀锋', '飞鸟集', '七夜雪', '最初的爱情 最后的仪式', '拆掉思维里的墙', '明朝那些事儿（陆）', '追风筝的人', '小王子', '围城', '解忧杂货店', '活着', '白夜行', '挪威的森林', '嫌疑人X的献身', '三体', '不能承受的生命之轻', '红楼梦', '梦里花落知多少', '达·芬奇密码', '看见', '百年孤独', '1988：我想和这个世界谈谈', '何以笙箫默', '平凡的世界（全三部）', '简爱', '哈利·波特与魔法石', '三体Ⅱ', '飘', '送你一颗子弹', '三体Ⅲ', '傲慢与偏见', '倾城之恋', '三重门', '杜拉拉升职记', '明朝那些事儿（壹）', '哈利·波特与阿兹卡班的囚徒', '目送', '情人', '哈利·波特与密室', '万历十五年', '我们仨', '幻城', '致我们终将逝去的青春', '狼图腾', '微微一笑很倾城', '莲花', '哈利·波特与火焰杯', '边城', '月亮和六便士', '向左走·向右走', '穆斯林的葬礼', '从你的全世界路过', '天龙八部', '放学后', '哈利·波特与混血王子

In [14]:
with open('/kaggle/input/doubanbook-top250/stopwords.txt', 'r') as f:
    stop_words = [line.strip() for line in f]
print(len(stop_words))

1165


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

books = []
comments = []
for b, c in book_comments.items():
    books.append(b)
    comments.append(' '.join(c))
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(comments)
similarity_matrix = cosine_similarity(tfidf_matrix)
print(similarity_matrix[:5, :5], similarity_matrix.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'daren', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mayn', 'mightn', 'mon', 'mustn', 'needn', 'oughtn', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn(


[[1.         0.12268639 0.13782096 0.09936095 0.07679552]
 [0.12268639 1.         0.12971906 0.1025389  0.09683428]
 [0.13782096 0.12971906 1.         0.11494365 0.11565774]
 [0.09936095 0.1025389  0.11494365 1.         0.07941307]
 [0.07679552 0.09683428 0.11565774 0.07941307 1.        ]] (233, 233)


In [16]:
import numpy as np

book_name = '杜拉拉升职记'
book_idx = books.index(book_name)
recommend_indices = np.argsort(-similarity_matrix[book_idx])[1:11]
for idx in recommend_indices:
    print(f'《{books[idx]}》\t\t\t相似度 {similarity_matrix[book_idx, idx]:.4f}')

《杜拉拉2华年似水》			相似度 0.7592
《拆掉思维里的墙》			相似度 0.1435
《把时间当作朋友》			相似度 0.1202
《致我们终将逝去的青春》			相似度 0.1141
《如何阅读一本书》			相似度 0.1110
《达·芬奇密码》			相似度 0.1103
《遇见未知的自己》			相似度 0.1103
《时间旅行者的妻子》			相似度 0.1085
《失恋33天》			相似度 0.1084
《小时代1.0折纸时代》			相似度 0.1015


In [17]:
import numpy as np

# bm25算法实现，输入为评论列表集合，k,b为超参数。输出所有评论的bm25结果矩阵
# 输入：comments = [['a','b','c'],['a','b','d'],['a','b','e']]
# 其中bm25[0] = [0.0, 0.0, 0.0, 0.0, 0.0]表示第一个评论的bm25值
# 其中bm25[0][0] = 0.0表示a的bm25值为0.0
def bm25(comments, k=1.5, b=0.75):
    # 计算文档总数
    N = len(comments)
    # 初始化文档长度列表和词频字典
    doc_lengths = []
    word_doc_freq = {}
    doc_term_dict = [{} for _ in range(N)]

    for i, comment in enumerate(comments):
        # 记录文档长度
        doc_lengths.append(len(comment))
        unique_words = set()
        for word in comment:
            # 统计词频
            doc_term_dict[i][word] = doc_term_dict[i].get(word, 0) + 1
            unique_words.add(word)
        # 统计包含该词的文档数量
        for word in unique_words:
            word_doc_freq[word] = word_doc_freq.get(word, 0) + 1

    # 计算每个单词的平均文档长度
    avg_doc_len = sum(doc_lengths) / N

    # 构建词汇表
    vocabulary = list(word_doc_freq.keys())
    word_index = {word: idx for idx, word in enumerate(vocabulary)}

    # 构建文档 - 词频矩阵
    doc_term_matrix = np.zeros((N, len(vocabulary)))
    for i in range(N):
        for word, freq in doc_term_dict[i].items():
            idx = word_index.get(word)
            if idx is not None:
                doc_term_matrix[i, idx] = freq

    # 计算 idf 值
    idf_numerator = N - np.array([word_doc_freq[word] for word in vocabulary]) + 0.5
    idf_denominator = np.array([word_doc_freq[word] for word in vocabulary]) + 0.5
    idf = np.log(idf_numerator / idf_denominator)
    idf[idf_numerator <= 0] = 0  # 避免出现 nan 值

    # 计算 bm25 值
    doc_lengths = np.array(doc_lengths)
    bm25_matrix = np.zeros((N, len(vocabulary)))
    for i in range(N):
        tf = doc_term_matrix[i]
        bm25 = idf * (tf * (k + 1)) / (tf + k * (1 - b + b * doc_lengths[i] / avg_doc_len))
        bm25_matrix[i] = bm25

    # 根据原始评论顺序重新排列 bm25 值
    final_bm25_matrix = []
    for i, comment in enumerate(comments):
        bm25_comment = []
        for word in comment:
            idx = word_index.get(word)
            if idx is not None:
                bm25_comment.append(bm25_matrix[i, idx])
        final_bm25_matrix.append(bm25_comment)

    # 找到最长的子列表长度
    max_length = max(len(row) for row in final_bm25_matrix)
    # 填充所有子列表到相同的长度
    padded_matrix = [row + [0] * (max_length - len(row)) for row in final_bm25_matrix]
    # 转换为 numpy 数组
    final_bm25_matrix = np.array(padded_matrix)

    return final_bm25_matrix

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

books = []
comments = []
for b, c in book_comments.items():
    books.append(b)
    comments.append(c)
tfidf_matrix = bm25(comments)
similarity_matrix = cosine_similarity(tfidf_matrix)
print(similarity_matrix[:5, :5], similarity_matrix.shape)

book_name = '杜拉拉升职记'
book_idx = books.index(book_name)
recommend_indices = np.argsort(-similarity_matrix[book_idx])[1:11]
for idx in recommend_indices:
    print(f'《{books[idx]}》\t\t\t相似度 {similarity_matrix[book_idx, idx]:.4f}')

[[1.         0.36988009 0.30820447 0.36946186 0.28358742]
 [0.36988009 1.         0.33040334 0.35787864 0.30823497]
 [0.30820447 0.33040334 1.         0.30906108 0.33358904]
 [0.36946186 0.35787864 0.30906108 1.         0.29413391]
 [0.28358742 0.30823497 0.33358904 0.29413391 1.        ]] (233, 233)
《年华是无效信》			相似度 0.4018
《1995-2005夏至未至》			相似度 0.3948
《悲伤逆流成河》			相似度 0.3871
《幻城》			相似度 0.3814
《一座城池》			相似度 0.3800
《哈利·波特与魔法石》			相似度 0.3794
《草样年华》			相似度 0.3784
《当我们谈论爱情时我们在谈论什么》			相似度 0.3764
《一个人的好天气》			相似度 0.3753
《苏菲的世界》			相似度 0.3741
